In [1]:
import numpy as np
import cv2 # OpenCV biblioteka
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial import distance
import csv
import os

In [2]:
BIN_THRESHOLD = 100
CARD_MIN_HW = 300
CARD_MAX_HW = 2000
EPSILON = 0.05
SEPARATE_CROP_Y = 37
CARD_MIN_HW = 300
CARD_MAX_HW = 2000
EPSILON = 0.05
CROP_Y0 = 5
CROP_H = 72
CROP_X0 = 2
CROP_W = 22
SEPARATE_CROP_Y = 37
CROP_THRESHOLD = 130

In [3]:
%matplotlib inline 
matplotlib.rcParams['figure.figsize'] = 16,12

In [4]:
clubs = 0
diams = 0
hearts = 0
spades = 0
count_ranks = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [5]:
path = os.getcwd()
with open('train.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        line_count += 1
        rank = row["ï»¿rank"]
        suit = row["suit"]
        number = row["number"]
        img_file = "train_images/" + rank + "-" + suit + "-" + number + ".jpg"

        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_gs = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, img_bin = cv2.threshold(img_gs, 100,255,cv2.THRESH_BINARY)

        img1, contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        contours_card = []
        i = 0
        for contour in contours:
            center, size, angle = cv2.minAreaRect(contour)
            width, height = size
            epsilon = EPSILON * cv2.arcLength(contour,True)
            approx = cv2.approxPolyDP(contour,epsilon,True)
            if width > CARD_MIN_HW and width < CARD_MAX_HW and height > CARD_MIN_HW and height < CARD_MAX_HW and len(approx) == 4 and hierarchy[0][i][3] == -1:
                #print(width, height, len(approx), hierarchy[0][i])
                contours_card.append(contour) # ova kontura pripada bar-kodu
            i += 1

        trazena_kontura = contours_card[0]
        x,y,w,h = cv2.boundingRect(trazena_kontura)
        rect = cv2.minAreaRect(trazena_kontura)
        n1,n2,angle = rect
        
        epsilon = EPSILON * cv2.arcLength(trazena_kontura,True)
        approx = cv2.approxPolyDP(trazena_kontura,epsilon,True)
        pts = [[approx[0][0][0], approx[0][0][1]], [approx[1][0][0], approx[1][0][1]], [approx[2][0][0], approx[2][0][1]], [approx[3][0][0], approx[3][0][1]]]
        pts = np.float32(pts)

        temp_rect = np.zeros((4,2), dtype = "float32")

        xSorted = pts[np.argsort(pts[:, 0]), :]

        leftMost = xSorted[:2, :]
        rightMost = xSorted[2:, :]

        leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
        (tl, bl) = leftMost

        D = distance.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
        (br, tr) = rightMost[np.argsort(D)[::-1], :]

        if distance.euclidean(bl,br) >= distance.euclidean(bl,tl):
            temp_rect[0] = bl
            temp_rect[1] = tl
            temp_rect[2] = tr
            temp_rect[3] = br
        else:
            temp_rect[0] = tl
            temp_rect[1] = tr
            temp_rect[2] = br
            temp_rect[3] = bl

        maxWidth = 200
        maxHeight = 300

        dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
        M = cv2.getPerspectiveTransform(temp_rect,dst)
        warp = cv2.warpPerspective(img, M, (maxWidth, maxHeight))

        crop_warp = warp[CROP_Y0:CROP_Y0 + CROP_H, CROP_X0:CROP_X0 + CROP_W]
        crop_gs = cv2.cvtColor(crop_warp, cv2.COLOR_RGB2GRAY)
        ret, crop_bin = cv2.threshold(crop_gs, CROP_THRESHOLD,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        znak_crop = crop_bin[0:SEPARATE_CROP_Y, 0:CROP_W]
        boja_crop = crop_bin[SEPARATE_CROP_Y:CROP_H, 0:CROP_W]

        suit_save_path = os.path.join(path , 'suits', suit)
        tmp = 0
        if suit == "C":
            clubs += 1
            tmp = clubs
        elif suit == "D":
            diams += 1
            tmp = diams
        elif suit == "H":
            hearts += 1
            tmp = hearts
        elif suit == "S":
            spades += 1
            tmp = spades
        suit_save_path = os.path.join(suit_save_path, str(tmp) + ".jpg")
        print(suit_save_path)
        cv2.imwrite(suit_save_path, boja_crop)


        rank_save_path = os.path.join(path , 'ranks', rank)
        tmp = 0
        if rank == "A":
            count_ranks[12] += 1
            tmp = count_ranks[12]
        elif rank == "K":
            count_ranks[11] += 1
            tmp = count_ranks[11]
        elif rank == "Q":
            count_ranks[10] += 1
            tmp = count_ranks[10]
        elif rank == "J":
            count_ranks[9] += 1
            tmp = count_ranks[9]
        elif rank == "T":
            count_ranks[8] += 1
            tmp = count_ranks[8]
        else:
            count_ranks[int(rank)-2] += 1
            tmp = count_ranks[int(rank)-2]
        rank_save_path = os.path.join(rank_save_path, str(tmp) + ".jpg")
        print(rank_save_path)
        cv2.imwrite(rank_save_path, znak_crop)
print('Done!')

C:\Users\Boris\Soft Project\suits\C\1.jpg
C:\Users\Boris\Soft Project\ranks\2\1.jpg
C:\Users\Boris\Soft Project\suits\C\2.jpg
C:\Users\Boris\Soft Project\ranks\2\2.jpg
C:\Users\Boris\Soft Project\suits\D\1.jpg
C:\Users\Boris\Soft Project\ranks\2\3.jpg
C:\Users\Boris\Soft Project\suits\D\2.jpg
C:\Users\Boris\Soft Project\ranks\2\4.jpg
C:\Users\Boris\Soft Project\suits\H\1.jpg
C:\Users\Boris\Soft Project\ranks\2\5.jpg
C:\Users\Boris\Soft Project\suits\H\2.jpg
C:\Users\Boris\Soft Project\ranks\2\6.jpg
C:\Users\Boris\Soft Project\suits\S\1.jpg
C:\Users\Boris\Soft Project\ranks\2\7.jpg
C:\Users\Boris\Soft Project\suits\S\2.jpg
C:\Users\Boris\Soft Project\ranks\2\8.jpg
C:\Users\Boris\Soft Project\suits\C\3.jpg
C:\Users\Boris\Soft Project\ranks\3\1.jpg
C:\Users\Boris\Soft Project\suits\C\4.jpg
C:\Users\Boris\Soft Project\ranks\3\2.jpg
C:\Users\Boris\Soft Project\suits\D\3.jpg
C:\Users\Boris\Soft Project\ranks\3\3.jpg
C:\Users\Boris\Soft Project\suits\D\4.jpg
C:\Users\Boris\Soft Project\ranks\

C:\Users\Boris\Soft Project\suits\C\26.jpg
C:\Users\Boris\Soft Project\ranks\A\2.jpg
C:\Users\Boris\Soft Project\suits\D\25.jpg
C:\Users\Boris\Soft Project\ranks\A\3.jpg
C:\Users\Boris\Soft Project\suits\D\26.jpg
C:\Users\Boris\Soft Project\ranks\A\4.jpg
C:\Users\Boris\Soft Project\suits\H\25.jpg
C:\Users\Boris\Soft Project\ranks\A\5.jpg
C:\Users\Boris\Soft Project\suits\H\26.jpg
C:\Users\Boris\Soft Project\ranks\A\6.jpg
C:\Users\Boris\Soft Project\suits\S\25.jpg
C:\Users\Boris\Soft Project\ranks\A\7.jpg
C:\Users\Boris\Soft Project\suits\S\26.jpg
C:\Users\Boris\Soft Project\ranks\A\8.jpg
C:\Users\Boris\Soft Project\suits\C\27.jpg
C:\Users\Boris\Soft Project\ranks\2\9.jpg
C:\Users\Boris\Soft Project\suits\D\27.jpg
C:\Users\Boris\Soft Project\ranks\2\10.jpg
C:\Users\Boris\Soft Project\suits\H\27.jpg
C:\Users\Boris\Soft Project\ranks\2\11.jpg
C:\Users\Boris\Soft Project\suits\S\27.jpg
C:\Users\Boris\Soft Project\ranks\2\12.jpg
C:\Users\Boris\Soft Project\suits\C\28.jpg
C:\Users\Boris\Soft

C:\Users\Boris\Soft Project\suits\D\49.jpg
C:\Users\Boris\Soft Project\ranks\K\10.jpg
C:\Users\Boris\Soft Project\suits\H\49.jpg
C:\Users\Boris\Soft Project\ranks\K\11.jpg
C:\Users\Boris\Soft Project\suits\S\49.jpg
C:\Users\Boris\Soft Project\ranks\K\12.jpg
C:\Users\Boris\Soft Project\suits\C\50.jpg
C:\Users\Boris\Soft Project\ranks\K\13.jpg
C:\Users\Boris\Soft Project\suits\D\50.jpg
C:\Users\Boris\Soft Project\ranks\K\14.jpg
C:\Users\Boris\Soft Project\suits\H\50.jpg
C:\Users\Boris\Soft Project\ranks\K\15.jpg
C:\Users\Boris\Soft Project\suits\S\50.jpg
C:\Users\Boris\Soft Project\ranks\K\16.jpg
C:\Users\Boris\Soft Project\suits\C\51.jpg
C:\Users\Boris\Soft Project\ranks\A\9.jpg
C:\Users\Boris\Soft Project\suits\D\51.jpg
C:\Users\Boris\Soft Project\ranks\A\10.jpg
C:\Users\Boris\Soft Project\suits\H\51.jpg
C:\Users\Boris\Soft Project\ranks\A\11.jpg
C:\Users\Boris\Soft Project\suits\S\51.jpg
C:\Users\Boris\Soft Project\ranks\A\12.jpg
C:\Users\Boris\Soft Project\suits\C\52.jpg
C:\Users\Bor

In [6]:
clubs_file = os.path.join(path, 'suits', 'C', 'count.txt')
fh = open(clubs_file, 'w')
fh.write(str(clubs))
fh.close()
diams_file = os.path.join(path, 'suits', 'D', 'count.txt')
fh = open(diams_file, 'w')
fh.write(str(diams))
fh.close()
hearts_file = os.path.join(path, 'suits', 'H', 'count.txt')
fh = open(hearts_file, 'w')
fh.write(str(hearts))
fh.close()
spades_file = os.path.join(path, 'suits', 'S', 'count.txt')
fh = open(spades_file, 'w')
fh.write(str(spades))
fh.close()

for i in range(0, 8):
    file_name = os.path.join(path, 'ranks', str(i+2), 'count.txt')
    fh = open(file_name,'w')
    fh.write(str(count_ranks[i]))
    fh.close()
tens_file = os.path.join(path, 'ranks', 'T', 'count.txt')
fh = open(tens_file, 'w')
fh.write(str(count_ranks[8]))
fh.close()
jacks_file = os.path.join(path, 'ranks', 'J', 'count.txt')
fh = open(jacks_file, 'w')
fh.write(str(count_ranks[9]))
fh.close()
queens_file = os.path.join(path, 'ranks', 'Q', 'count.txt')
fh = open(queens_file, 'w')
fh.write(str(count_ranks[10]))
fh.close()
kings_file = os.path.join(path, 'ranks', 'K', 'count.txt')
fh = open(kings_file, 'w')
fh.write(str(count_ranks[11]))
fh.close()
aces_file = os.path.join(path, 'ranks', 'A', 'count.txt')
fh = open(aces_file, 'w')
fh.write(str(count_ranks[12]))
fh.close()